In [1]:
# Disable TensorFlow debugging info and warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # 2: Info and warnings not displayed 

In [2]:
import tensorflow as tf
import numpy as np
import skimage.io
import matplotlib.pyplot as plt
import pathlib
import tqdm
import annotation, misc, hyspec_cnn, hyspec_io

In [3]:
# Disable GPUs
tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

In [4]:
# Paths
tiles_dir = pathlib.Path('/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles/TrainingAndValidationDatasets')
json_file = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles/tile_classes.json'
merged_dataset_path = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles/MergedDatasetNGT'
json_file_merged = '/media/mha114/Massimal/Larvik_Olberg/Hyperspectral/20210825/OlbergAreaS/4b_Rad_Georef_SGC_Tiles/tile_classes_merged_NGT.json'

In [5]:
# Read JSON with class names and labels
class_dict = annotation.read_class_dict(json_file)

In [6]:
# Load and merge datasets
dataset_dirs = [x for x in tiles_dir.glob('*/') if x.is_dir()]
merged_dataset = tf.data.experimental.load(str(dataset_dirs[0]))
for ds_dir in dataset_dirs[1:]:
    merged_dataset = merged_dataset.concatenate(tf.data.experimental.load(str(ds_dir)))

In [7]:
# Check how many tiles there are
n_tiles = merged_dataset.cardinality().numpy()
print(n_tiles)

459


In [8]:
def merge_nft_labels(image,labels):
    merged_label_indices = tf.constant([0,1,2,3,4,5,2,4,5])  # 6 = seagrass, 7 = rockweed, 8 = other
    new_labels = tf.gather(merged_label_indices,tf.cast(labels,tf.int32))
    return image, new_labels

In [9]:
dataset_ngt_labels_merged = merged_dataset.map(merge_nft_labels)

In [10]:
# Save dataset with labels merged
tf.data.experimental.save(dataset_ngt_labels_merged,merged_dataset_path)

In [11]:
# Save new class dict
class_dict_merged = {key:value for key,value in class_dict.items() if value<=5}
annotation.save_class_dict(class_dict_merged, json_file_merged)